In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('final_cleaned2.csv')

/Users/ruziheng/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,3,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# with regular expressions
# https://medium.com/mlearning-ai/elon-musks-twitter-sentiment-analysis-with-transformers-hugging-face-roberta-49b9e61b1433
# https://www.kaggle.com/code/ludovicocuoghi/twitter-sentiment-analysis-with-bert-roberta/notebook
# https://stackoverflow.com/questions/44027943/python-regex-to-remove-emails-from-string

def clean_re(tweet):
    # remove username
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    # remove links
    tweet = re.sub(r'http\S+', '', tweet) 
    tweet = re.sub(r'bit.ly/\S+', '', tweet) 
    tweet = tweet.strip('[link]') 
    # remove email address
    tweet = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", tweet)
    # remove special characters
    tweet = re.sub('([_]+)', "", tweet)
    tweet = tweet.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower()
    # remove links
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    # remove non-ascii characters
    tweet = re.sub(r'[^\x00-\x7f]',r'', tweet)
    # remove multiple spaces
    tweet = re.sub("\s\s+" , " ", tweet)
    # remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet

In [ ]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
data['text2'] = data['text2'].astype(str)
data['text2'] = data.text2.apply(func = clean_re)
data['text2'] = data['text2'].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha()))

In [3]:
peaceful = ['CA', 'FI', 'GB', 'NO', 'IE', 'AU', 'SG', 'FR']
data['peacefulness'] = data.country_code.apply(lambda x: 1 if (x in peaceful) else 0)
data = data[data['text2'].notna()]
peaceful = data.loc[data['peacefulness'] == 1]
peaceful = peaceful[['text2']].copy()
nonpeaceful = data.loc[data['peacefulness'] == 0]
nonpeaceful = nonpeaceful[['text2']].copy()

In [ ]:
high_corpus = [' '.join(peaceful['text2'])]
print(len(high_corpus[0].split()))

In [ ]:
low_corpus = [' '.join(nonpeaceful['text2'])]
print(len(low_corpus[0].split()))

In [4]:
tight = ['restrain', 'prevent', 'comply', 'constrain', 'uniformity', 'adhere', 'enforce', 'proscribe', 'abide',
         'dictate', 'circumscribe', 'impose', 'uphold', 'discourage', 'compel', 'forbid', 'confine', 'govern', 
         'prohibit', 'preclude']
loose = ['allow', 'freedom', 'create', 'variability', 'autonomy', 'openness', 'leeway', 'flexibility', 
         'broadmindedness', 'transformatory', 'customize', 'subjectivities', 'modify', 'limitless', 'empower',   
         'adaptiveness', 'pluralism', 'personalize', 'encourage', 'diverse']

In [27]:
nonpeaceful['loose'] = [' '.join([item for item in x.split() if item in loose]) for x in nonpeaceful['text2']]
nonpeaceful['tight'] = [' '.join([item for item in x.split() if item in tight]) for x in nonpeaceful['text2']]

In [28]:
peaceful['loose'] = [' '.join([item for item in x.split() if item in loose]) for x in peaceful['text2']]
peaceful['tight'] = [' '.join([item for item in x.split() if item in tight]) for x in peaceful['text2']]

In [32]:
# https://stackoverflow.com/questions/46786211/counting-the-frequency-of-words-in-a-pandas-data-frame
pd.Series(' '.join(nonpeaceful.tight).split()).value_counts().sum()

97

In [31]:
pd.Series(' '.join(nonpeaceful.tight).split()).value_counts()

prevent       30
abide         11
uphold         8
discourage     8
dictate        6
forbid         6
enforce        6
govern         6
adhere         4
impose         4
comply         3
restrain       2
compel         2
prohibit       1
dtype: int64

In [33]:
pd.Series(' '.join(nonpeaceful.loose).split()).value_counts().sum()

587

In [23]:
pd.Series(' '.join(nonpeaceful.loose).split()).value_counts()

freedom        209
create         172
allow          125
encourage       27
empower         23
diverse         10
flexibility      8
customize        7
limitless        3
modify           3
dtype: int64

In [35]:
pd.Series(' '.join(peaceful.tight).split()).value_counts().sum()

79

In [36]:
pd.Series(' '.join(peaceful.tight).split()).value_counts()

prevent       20
govern        11
enforce       11
abide          7
uphold         6
dictate        5
impose         5
discourage     3
forbid         3
comply         3
compel         2
prohibit       1
restrain       1
adhere         1
dtype: int64

In [34]:
pd.Series(' '.join(peaceful.loose).split()).value_counts().sum()

365

In [26]:
pd.Series(' '.join(peaceful.loose).split()).value_counts()

create         122
freedom        111
allow           66
encourage       22
diverse         15
flexibility      8
autonomy         6
limitless        4
empower          4
customize        3
leeway           2
openness         2
dtype: int64